## Lets run some queries in BQ first

#### I want to create some obfuscated tables where contents and table purpose are not immediately obvious 

create or replace table metadata_generation.cc as
SELECT unique_key uk, 
case_number cn, 
date as dt, 
block b, 
iucr as i, 
primary_type as pt, 
description as de, 
location_description as ld, 
arrest as ar, 
domestic as d, 
beat as bt, 
district di, 
ward as wr, 
community_area as cs, 
fbi_code as fc, 
x_coordinate xc, 
y_coordinate yc, 
year yr, 
updated_on uo, 
latitude la, 
longitude lo, 
location ltn 
FROM `bigquery-public-data.chicago_crime.crime` ;

insert into metadata_generation.cc
SELECT unique_key uk, 
case_number cn, 
date as dt, 
block b, 
iucr as i, 
primary_type as pt, 
description as de, 
location_description as ld, 
arrest as ar, 
domestic as d, 
beat as bt, 
district di, 
ward as wr, 
community_area as cs, 
fbi_code as fc, 
x_coordinate xc, 
y_coordinate yc, 
year yr, 
updated_on uo, 
latitude la, 
longitude lo, 
location ltn 
FROM `bigquery-public-data.chicago_crime.crime` ;

create or replace table metadata_generation.ccagg as
SELECT count(distinct unique_key) cuk, 
extract(month from date) as m, 
extract(day from date) as d, 
extract(hour from date) as h, 
primary_type as pt, 
arrest as ar, 
domestic as dc, 
beat as bt, 
district di, 
community_area as cs, 
fbi_code as fc, 
year yr
FROM `bigquery-public-data.chicago_crime.crime` 
group by 
extract(month from date), 
extract(day from date), 
extract(hour from date), 
primary_type, 
arrest, 
domestic, 
beat, 
district, 
community_area, 
fbi_code, 
year;

create or replace table metadata_generation.crimes as
SELECT  *
FROM `bigquery-public-data.chicago_crime.crime` ;

insert into metadata_generation.cc
select * from metadata_generation.crimes;



In [5]:
!pip install pandas

  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.5 MB/s eta 0:00:00
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.0 MB/s eta 0:00:0000:0100:01
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [8]:
!pip install google-cloud-aiplatform


  Using cached google_cloud_aiplatform-1.59.0-py2.py3-none-any.whl.metadata (31 kB)
  Using cached google_api_core-2.19.1-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_auth-2.32.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached proto_plus-1.24.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached google_cloud_bigquery-3.25.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached google_cloud_resource_manager-1.12.4-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached shapely-2.0.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.0 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached cachetools-5.4.0-py3-none-any.whl.metadata (5.3 kB)
  

In [10]:
!pip install google-cloud-dataplex


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 3.8 MB/s eta 0:00:00a 0:00:01


In [15]:
!pip install google-cloud-datacatalog


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 kB 4.0 MB/s eta 0:00:00a 0:00:01


In [18]:
!pip install google-cloud-datacatalog-lineage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 1.6 MB/s eta 0:00:00a 0:00:01


In [15]:


import pkgutil
import re
import json
import pandas
from enum import Enum
import vertexai
from google.cloud import bigquery
from google.cloud import dataplex_v1
from google.cloud.dataplex_v1 import (
    GetDataScanRequest,
    ListDataScanJobsRequest,
    GetDataScanJobRequest,
)
from google.cloud import datacatalog_lineage_v1
from google.cloud.dataplex_v1.types.datascans import DataScanJob
from google.cloud.exceptions import NotFound
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

import random
from google.cloud import storage

bqclient = bigquery.Client()
datascanclient= dataplex_v1.DataScanServiceClient()
lineageclient = datacatalog_lineage_v1.LineageClient()

I0000 00:00:1721825015.232717 11092278 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721825015.792750 11092278 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721825016.143348 11092278 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721825016.521291 11092278 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721825016.524198 11092278 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721825016.889898 11092278 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
table = ('jsk-dataplex-demo-380508.metadata_generation.cc')


In [16]:
bq_table=bqclient.get_table(table)

In [17]:
schema_to_json = [{"name": field.name, "type": field.field_type} for field in bq_table.schema]

In [20]:
all_scans = datascanclient.list_data_scans(parent=f"projects/jsk-dataplex-demo-380508/locations/us-central1")

all_scans_dprofile_dquality=[]

for scan in data_scans:
    if scan.data.resource == "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/cc":
        all_scans_dprofile_dquality.append(scan.name)

In [86]:
all_scans_dprofile_dquality

['projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a4b968725-70c9-4fa5-8570-54f2c372505d',
 'projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a88851ca9-7fab-4659-9a4f-07e95d97bf84',
 'projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a91cf9906-c8ff-4b17-86df-aedbf3bd06cf',
 'projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a11524cd2-a898-4e37-937e-b72fe85146c8',
 'projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a5e4e8aa2-e3c2-45ec-8731-a0f6fc3c68e1']

In [34]:
#This gets kinda hard here - we get profiles jobs results but only for data Profiling
collected_profiles=[]
for scan in all_scans_dprofile_dquality:
    for job in datascanclient.list_data_scan_jobs(ListDataScanJobsRequest(
        parent=datascanclient.get_data_scan(
            GetDataScanRequest(name=scan)).name
            )):
        job = datascanclient.get_data_scan_job(request=GetDataScanJobRequest(name=job.name,view="FULL"))
        if job.state == DataScanJob.State.SUCCEEDED:
            if "dataProfileResult" in json.loads(dataplex_v1.types.datascans.DataScanJob.to_json(job)):
                collected_profiles.append(
                    json.loads(dataplex_v1.types.datascans.DataScanJob.to_json(job))["dataProfileResult"]
                )

In [91]:
collected_profiles[0]


{'rowCount': '2399953',
 'profile': {'fields': [{'name': 'uk',
    'type': 'INTEGER',
    'mode': 'NULLABLE',
    'profile': {'distinctRatio': 0.9264701589327194,
     'topNValues': [{'value': '13340474',
       'count': '3',
       'ratio': 1.2511103604448948e-06},
      {'value': '13356600', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13356070', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13354224', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13354056', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13354012', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13353851', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13352753', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13352385', 'count': '2', 'ratio': 8.340735736299299e-07},
      {'value': '13351251', 'count': '2', 'ratio': 8.340735736299299e-07}],
     'integerProfile': {'average': 7186595.161851974,
   

In [52]:
entity=datacatalog_lineage_v1.EntityReference()
entity.fully_qualified_name="bigquery:"+table
search_request=datacatalog_lineage_v1.SearchLinksRequest(
    parent="projects/jsk-dataplex-demo-380508/locations/us",
    target=entity)
links=lineageclient.search_links(request=search_request)

In [59]:
source_tables=[]
link_names=[]
for link in links:
    source_tables.append(link.source.fully_qualified_name)
    link_names.append(link.name)


In [62]:
lineage_processes_ids = [
                        process.process
                        for process in lineageclient.batch_search_link_processes(
                            request=datacatalog_lineage_v1.BatchSearchLinkProcessesRequest(
                                parent=f"projects/jsk-dataplex-demo-380508/locations/us",
                                links=link_names,
                            )
                        )
                    ]

In [74]:
queries=[]
for process in lineage_processes_ids:
                        details = lineageclient.get_process(request=datacatalog_lineage_v1.GetProcessRequest(name=process))
                        if "bigquery_job_id" in details.attributes:
                            queries.append(bqclient.get_job(details.attributes["bigquery_job_id"], location='us').query)

In [70]:
queries

['insert into metadata_generation.cc\nselect * from metadata_generation.crimes',
 'insert into metadata_generation.cc\nSELECT unique_key uk, \ncase_number cn, \ndate as dt, \nblock b, \niucr as i, \nprimary_type as pt, \ndescription as de, \nlocation_description as ld, \narrest as ar, \ndomestic as d, \nbeat as bt, \ndistrict di, \nward as wr, \ncommunity_area as cs, \nfbi_code as fc, \nx_coordinate xc, \ny_coordinate yc, \nyear yr, \nupdated_on uo, \nlatitude la, \nlongitude lo, \nlocation ltn \nFROM `bigquery-public-data.chicago_crime.crime`',
 'create or replace table metadata_generation.cc as\nSELECT unique_key uk, \ncase_number cn, \ndate as dt, \nblock b, \niucr as i, \nprimary_type as pt, \ndescription as de, \nlocation_description as ld, \narrest as ar, \ndomestic as d, \nbeat as bt, \ndistrict di, \nward as wr, \ncommunity_area as cs, \nfbi_code as fc, \nx_coordinate xc, \ny_coordinate yc, \nyear yr, \nupdated_on uo, \nlatitude la, \nlongitude lo, \nlocation ltn \nFROM `bigque

In [82]:
def ask_llm(prompt):
    vertexai.init(project='jsk-dataplex-demo-380508', location='us-central1')

    model = GenerativeModel('gemini-1.5-pro-001')

    generation_config = GenerationConfig(
        temperature=0,
        top_p=0.95,
        top_k=32,
        max_output_tokens=8192,
    )
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }
    responses = model.generate_content(
        prompt,
        generation_config=generation_config,
        stream=False,
    )
    return responses.text

## Lets test generation quality depending on input metadata

In [90]:
ask_llm("Knowing that this is the schema of the table "+json.dumps(schema_to_json)+" Please provide short general description of the table contents and its purpose. Do not explain columns but provide overall description ")

I0000 00:00:1721913751.376517 11092278 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


'This table seems to store location-based data, potentially related to events or points of interest. It likely includes geographical coordinates, timestamps, and various attributes possibly describing the nature, status, or characteristics of each entry. \n'

In [93]:
ask_llm("Knowing that this is the schema of the table "+json.dumps(schema_to_json)+" and that this is table profile information:"+ json.dumps(collected_profiles[0])+" Please provide short general description of the table contents and its purpose. Do not explain columns but provide overall description ")

I0000 00:00:1721913824.992887 11092278 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


'This table appears to contain detailed information about criminal incidents, potentially in the city of Chicago. It likely serves as a record of individual crime reports, including their location, type, time, and various characteristics. \n'

In [94]:
ask_llm("Knowing that this is the schema of the table "+json.dumps(schema_to_json)+" and that this is table profile information:"+ json.dumps(collected_profiles[0])+" and that these queries are used to populate the table:"+json.dumps(queries)+" Please provide short general description of the table contents and its purpose. Do not explain columns but provide overall description ")

I0000 00:00:1721913890.352580 11092278 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


'The table contains data on crimes reported in Chicago. It includes various details about each crime incident, such as its type, location, date and time, and whether an arrest was made. Its purpose is likely for analysis of crime trends and patterns in the city. \n'